In [1]:
from langchain_community.llms import GPT4All
from IPython.display import display, Markdown
llm_path = "C://Users/pogawal/AppData/Local/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf"

llm = GPT4All(
    model=llm_path
)

In [ ]:
from IPython.display import display, Markdown
answer = llm.invoke("what are the rules for clean ABAP?")

display(Markdown(answer))  
 

In [4]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import WebBaseLoader
gpt4all_embd = GPT4AllEmbeddings()
loader = WebBaseLoader("https://github.com/SAP/styleguides/blob/main/clean-abap/CleanABAP.md")

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [15]:

from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

# loader = WebBaseLoader("https://www.fielmann.it/de/ueber-uns/")


In [6]:
scraped_document = loader.scrape()
# query_result = gpt4all_embd.embed_query(scraped_document)

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# load vector store
index_folder = "C://Users/pogawal/AppData/Local/nomic.ai/GPT4All/"
print("loading indexes")
faiss_index = FAISS.load_local(index_folder, embeddings, allow_dangerous_deserialization=True)
print("index loaded")
gpt4all_path = llm_path

In [ ]:
from gpt4all import Embed4All
text = 'The quick brown fox jumps over the lazy dog'
# embedder = Embed4All(device='cpu')
embedder = Embed4All('nomic-embed-text-v1.f16.gguf')
output = embedder.embed(text)
print(output)

In [ ]:
from IPython.display import display, Markdown
question = "Tell me about error handling"
matched_docs = faiss_index.similarity_search(question, 4)
context = ""
for doc in matched_docs:
    context = context + doc.page_content + " \n\n "

display(Markdown(context))    

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.fielmann-group.com/ueber-uns/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [12]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

In [ ]:
from langchain.embeddings import GPT4AllEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
embeddings = GPT4AllEmbeddings()
index_fielmann = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [ ]:
from langchain.llms import GPT4All
mistral_openorca = "mistral-7b-openorca.gguf2.Q4_0.gguf"
llm_path = "C://Users/pogawal/AppData/Local/nomic.ai/GPT4All/" + mistral_openorca

llm = GPT4All(model=llm_path)
# question = "Was macht die Fielmann AG?"
# docs = vectorstore.similarity_search(question)
# print(docs[0])
query = "Was macht die Fielmann AG?"
response = index_fielmann.query(query, llm=llm)
display(Markdown(response))

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate



mistral_nous = "Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf"
falcon = "gpt4all-falcon-newbpe-q4_0.gguf"



# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

# Run
question = "Was macht die Fielmann AG?"
docs = vectorstore.similarity_search(question)
chain.invoke(docs)